In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from plot_utils import *
from preprocess_datasets import load_features_outcomes, load_features_labels, make_training_sets, DEFAULT_FEATURES
from run_models import run_survival_model, run_rf_model, eval_model, save_model, load_model

# Make training sets if they haven't been created yet

device = 'cmod'
dataset = 'random32'
#numeric_feats = ['ip', 'n_e', 'aminor', 'kappa', 'squareness', 'delta', 'li', 'Wmhd']
numeric_feats = DEFAULT_FEATURES

# TODO: list disruptive vs non-disruptive shots in each dataset
make_training_sets(device, dataset)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Training shots: 19
Test shots: 6
Validation shots: 7


In [3]:
from auton_survival.preprocessing import Preprocessor
# Load and preprocess training, test, validation sets
features_train, outcomes_train = load_features_outcomes(device, dataset+'_train', features=numeric_feats)
features_test, outcomes_test = load_features_outcomes(device, dataset+'_test', features=numeric_feats)
features_val, outcomes_val = load_features_outcomes(device, dataset+'_val', features=numeric_feats)

# The features should match the above
_, labels_train = load_features_labels(device, dataset+'_train', 0.15)
_, labels_test = load_features_labels(device, dataset+'_test', 0.15)
_, labels_val = load_features_labels(device, dataset+'_val', 0.15)

# Fit the imputer and scaler to the training data and transform the training, test, and validation data
preprocessor = Preprocessor(cat_feat_strat='ignore', num_feat_strat='mean')
transformer=preprocessor.fit(features_train, cat_feats=[], num_feats=numeric_feats, one_hot=True, fill_value=-1)

x_train = transformer.transform(features_train)
x_test = transformer.transform(features_test)
x_val = transformer.transform(features_val)


In [4]:
# Train a cph model and save it
cph_model = run_survival_model('cph', x_train, x_val, outcomes_train, outcomes_val)
save_model(cph_model, transformer, 'cph', device, dataset)

Saved model to models/cph_cmod_random32.pkl


In [5]:
# Train a random forest model and save it
rf_model = run_rf_model(x_train, x_val, labels_train, labels_val)
save_model(rf_model, transformer, 'rf', device, dataset)

Saved model to models/rf_cmod_random32.pkl


In [6]:
# Train a dcph model and save it
dcph_model = run_survival_model('dcph', x_train, x_val, outcomes_train, outcomes_val)
save_model(dcph_model, transformer, 'dcph', device, dataset)

100%|██████████| 50/50 [00:00<00:00, 74.96it/s]


Error in fitting model for parameters: {'bs': 100, 'layers': [100], 'learning_rate': 0.0001}


100%|██████████| 50/50 [00:00<00:00, 91.10it/s]


Error in fitting model for parameters: {'bs': 100, 'layers': [100], 'learning_rate': 0.001}


100%|██████████| 50/50 [00:00<00:00, 58.72it/s]


...

100%|██████████| 50/50 [00:00<00:00, 58.84it/s]


Error in fitting model for parameters: {'bs': 200, 'layers': [100, 100], 'learning_rate': 0.0001}


100%|██████████| 50/50 [00:00<00:00, 58.85it/s]

Error in fitting model for parameters: {'bs': 200, 'layers': [100, 100], 'learning_rate': 0.001}


ValueError: No models trained for dcph

In [7]:
# Train a random survival forest model and save it
rsf_model = run_survival_model('rsf', x_train, x_val, outcomes_train, outcomes_val)
save_model(rsf_model, transformer, 'rsf', device, dataset)


Saved model to models/rsf_cmod_random32.pkl


In [33]:
%%capture --no-display
# Train a dsm model and save it
dsm_model = run_survival_model('dsm', x_train, x_val, outcomes_train, outcomes_val)
save_model(dsm_model, transformer, 'dsm', device, dataset)

In [36]:
%%capture --no-display
# Train a dcm model and save it
dcm_model = run_survival_model('dcm', x_train, x_val, outcomes_train, outcomes_val)
save_model(dcm_model, transformer, 'dcm', device, dataset)

In [8]:
# Load a model
cph_model = load_model('cph', device, dataset)

Loaded model from models/cph_cmod_random100.pkl
